In [1]:
# Some imports
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from bs4 import BeautifulSoup as bs4
import requests
import urllib.request
import os
from multiprocessing import Process, Pool # SubProcessing
import time
import threading

In [2]:
# Create dataframe where we will save our details about cars
final_data = pd.DataFrame(columns = ["ID", "Manufacturer", "Model", "Category", "Mileage", "Gear box type", "Doors", "Wheel", "Color", "Interior color", "VIN", "Leather interior", "Price", "Customs"])

In [3]:
# Add appropriate columns
final_data

,ID,Manufacturer,Model,Category,Mileage,Gear box type,Doors,Wheel,Color,Interior color,VIN,Leather interior,Price,Customs


In [4]:
# current path
my_path = "C:/Users/bgoga/Desktop/computer vision"

# helping dic
my_dic = {0:'Model', 2:'Category', 5:'Mileage', 7:'Gear box type', 9:'Doors', 10:'Wheel', 11:'Color', 12:'Interior color', 14:'VIN'}

# search url where we can start.
search_url = "https://www.myauto.ge/en/s/00/0/00/00/00/00/00/cars?stype=0&currency_id=3&det_search=0&ord=7&category_id=m0&keyword="

In [5]:
# This method create lists of urls.
# Doesnot need this method anymore, couse we split urls, but still if needed here it is.
def createList(search_url, final_data):
    url_list, count = [], 0
    while(1):
        
        if count == 1:
            break
        request = requests.get(search_url)
        soup = bs4(request.text, 'lxml')
        list_url = soup.findAll("div", {"class": "car-name-left"})
        
        # loop over urls 
        for _ in range(0, len(list_url)):
            url = list_url[_].find('a', href=True)['href']
            url_list.append(url)
        
        try:
            search_url = soup.findAll("li", {"class": "pagination-li pag-next"})[0].find('a', href=True)['href']
        except:
            break    
        
        count += 1
    return url_list

In [6]:
# This method returns some additional informations
# ID, Price, Leather, Custom
def getAdditionalDetails(url, soup):
    # Take price
    price = soup.findAll("span", {"class": "car-price"})[1].text
    if price != 'Price negotiable':
        price = price.split(' ')[0]
        price = price.replace(',', '')
    
    # Take ID
    ID = soup.findAll("div", {"class": "detail-top-item"})[2].text
    
    # Take Leather
    leather = "1"
    if "fa-minus" in  soup.findAll("div", {"class": "th-value"})[9].findChildren("i" , recursive=False)[0].attrs['class']:
        leather = "0"
    
    # Take Custom
    custom = None
    try:
        custom = soup.findAll("div", {"class": "levy"})[0].find_all('span', {'data-id' : '1'})[0].text
        custom = custom.split(' ')[0]
        price = price.replace(',', '')
    except:
        custom = ""
        
    return ID, price, leather, custom

# Download thumbnail images 
def downloadImages(ID, soup):
    # create directory
    os.mkdir(ID)    
    
    # download specific images on ID folder path!
    other_imgs = soup.findAll("div", {"class": "thumbnail-image"})
    for ind, image in enumerate(other_imgs):
        urllib.request.urlretrieve(image.find('img')['src'], my_path + "//" + str(ID) +  "//{}.jpg".format(ind))


# Taking main table and parsing information    
def fillData(df, url, soup):
    my_list = []
    # column base case
    column_name = df.columns[0]
    
    # call additional Details
    ID, price, leather, custom = getAdditionalDetails(url, soup)
    
    # call download images
    downloadImages(ID, soup)
    
    my_list.append(ID)
    my_list.append(column_name.lstrip('Manufacturer').lstrip())
    df = df[column_name]
    
    # parse info
    for _ in my_dic.keys():
        my_list.append(df[_][len(my_dic[_]):].lstrip())
    
    my_list.append(leather)
    my_list.append(price)
    my_list.append(custom)
    return my_list

# Method to iterate one file.
def iterateOneFile(url):
    # get html
    html_source = requests.get(url).content
    
    # parse
    soup = BeautifulSoup(html_source,'lxml')
    new_table = soup.find_all('table')
    table = new_table[0]
    df = pd.read_html(str(table), header=0)[0]
    
    # take list of information
    return fillData(df, url, soup)
    

In [5]:
# Final method!!!
def final_method(lines, lists):
    final_list, lines, t= [], lines[1], 0
    
    for _ in lines:
        try:
            my_list = iterateOneFile(_)
            lists.append(pd.Series(my_list, index=final_data.columns))
        except:
            pass

In [8]:
# open text file to take my part of urls
with open('beqa.txt') as f:
    lines = f.read().splitlines()

In [9]:
# divide Links to 7 part for threading!
pol_len, links_arr = 7, []
chunks = len(lines) // pol_len

# divide data
for _ in range(0, pol_len):
    if _ == pol_len - 1:
        links_arr += [[_, lines[chunks * _ :]]]
    else:
        links_arr += [[_, lines[chunks * _ : chunks * (_ + 1)]]]


In [10]:
import threading

In [13]:
# our main to call threads
if __name__ == "__main__":
    in_t = time.time()
    
    workers, lists = [], []
    
    # run 7 thread and  enjoy!
    for _ in range(0, 7):
        workers.append(threading.Thread(target=final_method, args=(links_arr[_], lists)))
       
    for _ in range(0, 7):
        workers[_].start()
    
    for _ in range(0, 7):
        workers[_].join()
    in_out = time.time()    
    print(in_out - in_t)

5154.76819562912


In [14]:
lists

[ID                           45628667
 Manufacturer                   TOYOTA
 Model                          Tacoma
 Category                        Sedan
 Mileage                     397915 km
 Gear box type               Automatic
 Doors                                
 Wheel                      Left wheel
 Color                           White
 Interior color                       
 VIN                 5TFJX4GN4DX026901
 Leather interior                    1
 Price                            9100
 Customs                           834
 dtype: object,
 ID                    45788844
 Manufacturer            TOYOTA
 Model                    RAV 4
 Category                  Jeep
 Mileage               30402 km
 Gear box type         Variator
 Doors                      4/5
 Wheel               Left wheel
 Color                     Blue
 Interior color           Black
 VIN                           
 Leather interior             0
 Price                    15000
 Customs              

In [15]:
final_data = final_data.append(lists , ignore_index=True)

In [16]:
final_data

,ID,Manufacturer,Model,Category,Mileage,Gear box type,Doors,Wheel,Color,Interior color,VIN,Leather interior,Price,Customs
0,45628667,TOYOTA,Tacoma,Sedan,397915 km,Automatic,,Left wheel,White,,5TFJX4GN4DX026901,1,9100,834
1,45788844,TOYOTA,RAV 4,Jeep,30402 km,Variator,4/5,Left wheel,Blue,Black,,0,15000,518
2,45653468,HONDA,Insight,Hatchback,210758 km,Automatic,4/5,Left wheel,Silver,,JHMZE2H57AS029004,1,800,574
3,45731431,KIA,Optima,Sedan,131040 km,Tiptronic,4/5,Left wheel,White,Black,KNAGM4AD8D5052655,0,5500,751
4,45771182,LEXUS,ES 300,Sedan,135500 km,Tiptronic,4/5,Left wheel,White,Black,,1,13500,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16830,45653518,HONDA,Civic,Jeep,146403 km,Automatic,4/5,Left wheel,White,,JHMFB4F2XCS000292,1,925,503
16831,45653512,HONDA,Insight,Hatchback,161328 km,Automatic,4/5,Left wheel,White,,JHMZE2H73BS007861,1,Price negotiable,490
16832,45653503,HONDA,Insight,Hatchback,235563 km,Automatic,4/5,Left wheel,Grey,,JHMZE2H53AS029226,1,Price negotiable,574
16833,45653490,HONDA,Insight,Hatchback,497830 km,Automatic,4/5,Left wheel,Black,,JHMZE2H75AS040200,1,Price negotiable,574


In [17]:
final_data.to_csv("car_info_beqa.csv", index=False)